# Mapa das eleições municipais - Tutorial

Esse documento irá apresentar um passo-a-passo de como gerar um arquivo final contendo a quantidade de votos de cada candidato por local de votação. 

Inicialmente, vamos importar as bibliotecas básicas necessárias para nossa análise.

In [ ]:
import pandas as pd
import numpy as np

Vamos agora ler o arquivo csv extraído da [base de dados TSE](https://dadosabertos.tse.jus.br/dataset/resultados-2024). Utilizaremos os arquivos de votos por seção eleitoral. Lembrando que, para o TSE, existe apenas zona e seção, sendo o local de votação uma abstração que iremos criar a partir desse arquivo.

Para nossa análise, estamos usando dados do Estado do Rio de Janeiro, mas você pode substituir por qualquer outro que desejar.

Importante observar que o encoding do arquivo está em ISO-8859-1 (latin-1), sendo necessário especificar nos argumentos do read_csv. Para garantir também que valores como número do candidato não sejam convertidos automaticamente, vamos forçar a leitura dessas colunas em string, convertendo posterior o quantitativo de votos em integer.

In [ ]:
# Load CSV file
file_path = 'votacao_secao_2024_RJ.csv'
df = pd.read_csv(file_path, sep=';', encoding='ISO-8859-1', dtype=str)

# Convert QT_VOTOS to integer
df['QT_VOTOS'] = df['QT_VOTOS'].astype(int)

Agora, vamos filtrar apenas os dados do município carioca, usando como base o código municipal. Importante observar que o TSE possui numeração de municípios distinta da utilizada pelo IBGE, não devendo ser confundida uma com a outra. Você pode descobrir o código do município desejado acessando [esse pdf](https://www.tse.jus.br/++theme++justica_eleitoral/pdfjs/web/viewer.html?file=https://www.tse.jus.br/comunicacao/noticias/arquivos/tse-eleitorado-uf-2020/@@download/file/TSE-Perfil%20Eleitorado%20munic%C3%ADpios%202020.pdf).

In [ ]:
# Filter rows where CD_MUNICIPIO (col 14) == 60011
df_filtered = df[df['CD_MUNICIPIO'] == '60011']

Agora, iremos calcular o número de votos válidos em cada local de votação para podermos, ao final, calcular a proporção daquele candidato. Isso alivia os recursos tomados do lookerstudio.
Importante observar que alguns locais compartilham o mesmo endereço, razão pela qual é necessário incluir o nome do local também. Além disso, precisamos separar por cargos, visto que, do contrário, incluiríamos votos de prefeito no cálculo de vereador e vice-versa.
A proporção está sendo calculado apenas com base nos votos válidos, ou seja, tirando os nulos e brancos, qual a proporção percentual de votos daquele candidato naquela seção dentre os candidatos daquele cargo?

Em seguida, iremos agrupar por candidato e local para somar o total de votos de todas as seções eleitorais daquele local.

Por fim, adicionaremos uma string no campo de endereço especificando que se trata do Rio de Janeiro, para o mapa do dashboard identificar corretamente.

In [ ]:
df_filtered['ID_ENDERECO'] = df_filtered['NM_LOCAL_VOTACAO'] + ', ' + df_filtered['DS_LOCAL_VOTACAO_ENDERECO'] + ' / ' + df_filtered['DS_CARGO'] 

# Create a table with the sum by ID_ENDERECO of QT_VOTOS where NR_VOTAVEL is not 95 or 96
valid_votes_df = df_filtered[~df_filtered['NR_VOTAVEL'].isin(['95', '96'])].groupby('ID_ENDERECO')['QT_VOTOS'].sum().reset_index()
valid_votes_df.rename(columns={'QT_VOTOS': 'TOTAL_VOTOS_VALIDOS'}, inplace=True)

# Group by SQ_CANDIDATO (col 24) and ID_ENDERECO
# and aggregate by summing QT_VOTOS (col 22)
agg_df = df_filtered.groupby(['SQ_CANDIDATO', 'ID_ENDERECO']).agg(
    {
        'QT_VOTOS': 'sum',
        'NR_VOTAVEL': 'first',  # Get any value, since it's the same for all rows with the same SQ_CANDIDATO
        'NM_VOTAVEL': 'first',
        'DS_LOCAL_VOTACAO_ENDERECO': 'first',  # Same for all rows with the same NR_LOCAL_VOTACAO
        'NM_LOCAL_VOTACAO': 'first',
        'DS_CARGO': 'first'
    }
).reset_index()

# Update DS_LOCAL_VOTACAO_ENDERECO
agg_df['DS_LOCAL_VOTACAO_ENDERECO'] = agg_df['DS_LOCAL_VOTACAO_ENDERECO'] + ', Rio de Janeiro - RJ, Brasil'

Agora, iremos ler os dados de candidatura, também extraídos da [base de dados do TSE](https://dadosabertos.tse.jus.br/dataset/candidatos-2024). Isso é necessário porque não temos nem o nome de urna do candidato nem a sigla do partido no arquivo anterior, dificultando sua identificação. Vamos criar uma tabela com os valores únicos do nome do candidato e seu partido. Em seguida, iremos gerar uma coluna com o nome final, contendo o nome de urna, número e sigla do partido.

Um fato interessante é que, para vereador, é possível votar em legenda. Como não temos um candidato no arquivo de candidatos representando as legendas, mas, a partir do arquivo de candidatos, podemos gerar uma lista relacional de número do partido e sua respectiva sigla.

Dessa forma, realizamos um left join (merge), primeiro com base no  sequencial do candidato, para pegar seu nome de urna (coluna VOTAVEL), depois com base nos dois primeiros números de urna, para pegar o partido e sua sigla.

Por fim, preenchemos os valores para Brancos, Nulos e Voto em Legenda.

In [ ]:
# Load candidate information from consulta_cand_2024_RJ.csv
candidate_file_path = 'consulta_cand_2024_RJ.csv'
candidate_df = pd.read_csv(candidate_file_path, sep=';', encoding='ISO-8859-1', dtype=str)

# Filter rows where CD_MUNICIPIO == 60011
candidate_df_filtered = candidate_df[candidate_df['SG_UE'] == '60011']

# Get distinct values of SQ_CANDIDATO, NM_URNA_CANDIDATO, NR_CANDIDATO, and SG_PARTIDO
candidate_info_df = candidate_df_filtered[['SQ_CANDIDATO', 'NM_URNA_CANDIDATO', 'NR_CANDIDATO', 'SG_PARTIDO', 'NM_PARTIDO']].drop_duplicates()

# Create VOTAVEL column
candidate_info_df['VOTAVEL'] = candidate_info_df['NM_URNA_CANDIDATO'] + ' - ' + candidate_info_df['NR_CANDIDATO'] + ' (' + candidate_info_df['SG_PARTIDO'] + ')'

# Create a separate df with distinct values of NR_PARTIDO and SG_PARTIDO
partido_info_df = candidate_df_filtered[['NR_PARTIDO', 'SG_PARTIDO']].drop_duplicates()
partido_info_df['PARTIDO'] = partido_info_df['NR_PARTIDO'] + ' - ' + partido_info_df['SG_PARTIDO']
partido_info_df = partido_info_df.drop(columns=['SG_PARTIDO'])

# Merge agg_df with candidate_info_df on SQ_CANDIDATO
agg_df = agg_df.merge(candidate_info_df[['SQ_CANDIDATO', 'VOTAVEL']], on='SQ_CANDIDATO', how='left')

# Left join with partido_info_df by matching the first two characters of NR_VOTAVEL with NR_PARTIDO
agg_df = agg_df.merge(partido_info_df, left_on=agg_df['NR_VOTAVEL'].str[:2], right_on='NR_PARTIDO', how='left')

# Fill NaN values in VOTAVEL column
agg_df['VOTAVEL'] = agg_df.apply(lambda row: 'BRANCO' if pd.isna(row['VOTAVEL']) and row['NR_VOTAVEL'] == '95' else ('NULO' if pd.isna(row['VOTAVEL']) and row['NR_VOTAVEL'] == '96' else ('Legenda: ' + str(row['PARTIDO']) if pd.isna(row['VOTAVEL']) else row['VOTAVEL'])), axis=1)

# Fill empty PARTIDO values with NM_VOTAVEL
agg_df['PARTIDO'] = agg_df.apply(lambda row: 'BRANCO/NULO' if row['NR_VOTAVEL'] in ['95', '96'] else row['NM_VOTAVEL'] if pd.isna(row['PARTIDO']) or row['PARTIDO'] == '' else row['PARTIDO'], axis=1)

Para finalizar, realizamos um último merge com o total de votos válidos naquele local para aquele cargo e calculamos a proporção de cada candidato com base nisso. Para votos brancos e nulos, mantemos o valor de 0 visto que a proporção é apenas dentre os votos válidos.

Agora basta gerar o arquivo csv e importar no LookerStudio.

In [ ]:
# Left join with valid_votes_df to add TOTAL_VOTOS_VALIDOS
agg_df = agg_df.merge(valid_votes_df, on='ID_ENDERECO', how='left')

# Calculate % de votos válidos do local
agg_df['% de votos válidos do local'] = np.where(
    agg_df['NR_VOTAVEL'].isin(['95', '96']),
    0,
    round((agg_df['QT_VOTOS'].astype(float) / agg_df['TOTAL_VOTOS_VALIDOS'].astype(float)) * 100, 2)
)

# Drop the original NR_VOTAVEL, NM_VOTAVEL, ID_ENDERECO, and TOTAL_VOTOS_VALIDOS columns
agg_df = agg_df.drop(columns=['NR_VOTAVEL', 'NM_VOTAVEL', 'ID_ENDERECO', 'TOTAL_VOTOS_VALIDOS', 'NR_PARTIDO'])

# Save the final dataframe to a new CSV file
output_file_path = 'final_output.csv'
agg_df.to_csv(output_file_path, index=False)

print(f"Final file saved to {output_file_path}")

<hr>
<sub><p xmlns:cc="http://creativecommons.org/ns#" xmlns:dct="http://purl.org/dc/terms/"><a property="dct:title" rel="cc:attributionURL" href="https://jncoe.github.io/mapa_eleicoes/Tutorial.html">Mapa das eleições municipais - Tutorial</a> por <a rel="cc:attributionURL dct:creator" property="cc:attributionName" href="https://jncoe.github.io/">Jonas Coelho</a> está licenciada sob <a href="https://creativecommons.org/licenses/by-nc-sa/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">CC BY-NC-SA 4.0<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1" alt=""><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1" alt=""><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/nc.svg?ref=chooser-v1" alt=""><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1" alt=""></a></p></sub>